In [1]:
import numpy as np
import random
import sys
import io
import os
from mb_utils import *
%matplotlib inline
import pickle

import pandas as pd

c:\users\lukewolc\appdata\local\programs\python\python37\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
Using TensorFlow backend.


In [6]:
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape, Flatten
from keras.optimizers import Adam
from keras.utils import np_utils
import keras.backend as K
from time import time

In [3]:
from importlib import reload
import mb_utils

In [4]:
reload(mb_utils)
from mb_utils import *

In [5]:
Tx = 2754 #5511 # The number of time steps input to the model from the spectrogram
n_freq = 101 # Number of frequencies input to the model at each time step of the spectrogram
Ty = 1375 # The number of time steps in the output of our model

# model training

In [7]:
# load training features and labels
file = open('XY_train/X_1000_5sec_v0.pkl', 'rb')
X_training = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v0.pkl', 'rb')
y_training = pickle.load(file)
file.close()

# load validation features and labels
file = open('XY_val/X_200_5sec_50_50_100.pkl', 'rb')
X_val = pickle.load(file)
file.close()

file = open('XY_val/y_200_5sec_50_50_100.pkl', 'rb')
y_val = pickle.load(file)
file.close()

# one-hot encode labels
y_training_oh = encode_and_oh_labels(y_training)
y_val_oh = encode_and_oh_labels(y_val)

print('Size of X_training: {}'.format(X_training.shape))
print('Size of X_val: {}'.format(X_val.shape))

Size of X_training: (1000, 2754, 101)
Size of X_val: (200, 2754, 101)


In [ ]:
# load training features and labels for 4000 over 4 files

#v0
file = open('XY_train/X_1000_5sec_v0.pkl', 'rb')
X_training_v0 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v0.pkl', 'rb')
y_training_v0 = pickle.load(file)
file.close()

#v1
file = open('XY_train/X_1000_5sec_v1.pkl', 'rb')
X_training_v1 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v1.pkl', 'rb')
y_training_v1 = pickle.load(file)
file.close()

#v2
file = open('XY_train/X_1000_5sec_v2.pkl', 'rb')
X_training_v2 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v2.pkl', 'rb')
y_training_v2 = pickle.load(file)
file.close()

#v3
file = open('XY_train/X_1000_5sec_v3.pkl', 'rb')
X_training_v3 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v3.pkl', 'rb')
y_training_v3 = pickle.load(file)
file.close()

X_training = np.concatenate([X_training_v0, X_training_v1, X_training_v2, X_training_v3])
y_training = np.concatenate([y_training_v0, y_training_v1, y_training_v2, y_training_v3])

del(X_training_v0)
del(X_training_v1)
del(X_training_v2)
del(X_training_v3)

# load validation features and labels
file = open('XY_val/X_200_5sec_50_50_100.pkl', 'rb')
X_val = pickle.load(file)
file.close()

file = open('XY_val/y_200_5sec_50_50_100.pkl', 'rb')
y_val = pickle.load(file)
file.close()

# one-hot encode labels
y_training_oh = encode_and_oh_labels(y_training)
y_val_oh = encode_and_oh_labels(y_val)

print('Size of X_training: {}'.format(X_training.shape))
print('Size of X_val: {}'.format(X_val.shape))

In [8]:
label_decoder_dict = {0:'empty', 1:'enough', 2:'not_enough'}

In [9]:
# shuffle training data explicitly
np.random.seed(134)
perm = np.random.permutation(len(X_training))

X_training = X_training[perm]
y_training = y_training[perm]
y_training_oh = y_training_oh[perm]

In [93]:
def model(input_shape):
    X_input = Input(shape = input_shape)
    
    X = Conv1D(filters=196, kernel_size=15, strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    #X = Dropout(0.0)(X)
    
    X = Bidirectional(LSTM(units=128))(X)
    X = Dense(3, activation='softmax')(X)

    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [10]:
def gru_model(input_shape):
    X_input = Input(shape = input_shape)
    
    X = Conv1D(filters=196, kernel_size=15, strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.5)(X)

    X = GRU(units=128, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    X = BatchNormalization()(X)
    
    X = GRU(units=128, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    X = BatchNormalization()(X)
    X = Dropout(0.5)(X)
    
    X = TimeDistributed(Dense(1, activation='relu'))(X)
    X = Reshape((685,))(X)
    X = Dense(3, activation='softmax')(X)
    
    model = Model(inputs=X_input, outputs=X)
    return model

In [22]:
def gru_model2(input_shape):
    X_input = Input(shape = input_shape)
    
    X = Conv1D(filters=196, kernel_size=15, strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.5)(X)

    X = GRU(units=128, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    X = BatchNormalization()(X)
    
    X = GRU(units=128, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    X = BatchNormalization()(X)
    X = Dropout(0.5)(X)
    
#    X = TimeDistributed(Dense(1, activation='relu'))(X)
#    X = Reshape((685,))(X)
    X = Flatten()(X)
    X = Dense(3, activation='softmax')(X)
    
    model = Model(inputs=X_input, outputs=X)
    return model

In [23]:
K.clear_session()

In [24]:
model = gru_model2(input_shape = (Tx, n_freq))

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2754, 101)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 685, 196)          297136    
_________________________________________________________________
batch_normalization_1 (Batch (None, 685, 196)          784       
_________________________________________________________________
activation_1 (Activation)    (None, 685, 196)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 685, 196)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 685, 128)          124800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 685, 128)          0         
__________

In [26]:
version = '{}'.format(int(time()%1e7))
tb_path = 'tensorboard_logs/{}'.format(version)
tensorboard = TensorBoard(log_dir=tb_path, histogram_freq=1)
print('version: {}'.format(version))

#opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

version: 4055836


In [27]:
model.fit(X_training, y_training_oh, batch_size=32, epochs=10, shuffle=True,
         validation_data = [X_val, y_val_oh], callbacks = [tensorboard])

Train on 1000 samples, validate on 200 samples
Epoch 1/10
1000/1000 [==============================] - 44s 44ms/step - loss: 1.3114 - acc: 0.6730 - val_loss: 0.2660 - val_acc: 0.9050
Epoch 2/10
1000/1000 [==============================] - 42s 42ms/step - loss: 0.5201 - acc: 0.8940 - val_loss: 0.7793 - val_acc: 0.9200
Epoch 3/10
1000/1000 [==============================] - 42s 42ms/step - loss: 0.2958 - acc: 0.9460 - val_loss: 1.7733 - val_acc: 0.8150
Epoch 4/10
1000/1000 [==============================] - 42s 42ms/step - loss: 0.2149 - acc: 0.9670 - val_loss: 0.5398 - val_acc: 0.9500
Epoch 5/10
1000/1000 [==============================] - 42s 42ms/step - loss: 0.2514 - acc: 0.9660 - val_loss: 0.4498 - val_acc: 0.9450
Epoch 6/10
1000/1000 [==============================] - 43s 43ms/step - loss: 0.1219 - acc: 0.9760 - val_loss: 0.4986 - val_acc: 0.9450
Epoch 7/10
1000/1000 [==============================] - 42s 42ms/step - loss: 0.0435 - acc: 0.9910 - val_loss: 0.3022 - val_acc: 0.9750
E

In [34]:
# get predictions on validation set
labels = y_val
preds = model.predict(X_val)
preds_text = [label_decoder_dict[v] for v in np.argmax(preds, axis=1)]

In [31]:
# get predictions on training set
labels = y_training[:200]
preds = model.predict(X_training[:200])
preds_text = [label_decoder_dict[v] for v in np.argmax(preds, axis=1)]

In [35]:
results = pd.DataFrame({'labels':labels, 'preds':preds_text})

In [36]:
pd.crosstab(results['labels'], results['preds'])

preds,empty,enough,not_enough
labels,,,
empty,100,0,0
enough,0,50,0
not_enough,0,6,44


In [37]:
model.save('trained_models/{}.h5'.format(version))

In [38]:
version

'4055836'

####  version notes

- 1st: good data, 1000.  perfect on train and test but overfitting. randomizing where words appear in clip but only using a few dozen clips.
- 2nd: good data, 4000. perfect on train and test but overfitting. same as above.
- `4055836`: train on 1000, after (4x4) data augmentation on each of enoughs, not_enoughs, empty. high 90s, except for predicting some not_enoughs as enoughs.